# ООП. Абстрактный класс. Декомпозиция программы.

Time is undefined for this lesson, because access to lessons was cut short while I was doing this

In [ ]:
from abc import ABC, abstractmethod

In [1]:
import math
from abc import ABC, abstractmethod


class Base(ABC):
    def __init__(self, data, result):
        self.data = data
        self.result = result

    def get_answer(self):
        return [int(x >= 0.5) for x in self.data]

    @abstractmethod
    def get_score(self):
        pass

    @abstractmethod
    def get_loss(self):
        pass


class A(Base):
    def get_score(self):
        ans = self.get_answer()
        return sum([int(x == y) for (x, y) in zip(ans, self.result)]) \
            / len(ans)

    def get_loss(self):
        return sum(
            [(x - y) * (x - y) for (x, y) in zip(self.data, self.result)])


class B(Base):
    def get_loss(self):
        return -sum([
            y * math.log(x) + (1 - y) * math.log(1 - x)
            for (x, y) in zip(self.data, self.result)
        ])

    def get_pre(self):
        ans = self.get_answer()
        res = [int(x == 1 and y == 1) for (x, y) in zip(ans, self.result)]
        return sum(res) / sum(ans)

    def get_rec(self):
        ans = self.get_answer()
        res = [int(x == 1 and y == 1) for (x, y) in zip(ans, self.result)]
        return sum(res) / sum(self.result)

    def get_score(self):
        pre = self.get_pre()
        rec = self.get_rec()
        return 2 * pre * rec / (pre + rec)


class C(Base):
    def get_score(self):
        ans = self.get_answer()
        return sum([int(x == y) for (x, y) in zip(ans, self.result)]) \
            / len(ans)

    def get_loss(self):
        return sum([abs(x - y) for (x, y) in zip(self.data, self.result)])

## Task 2
Refactor *THIS*

In [ ]:
from typing import Self, Optional
import random
import pygame

Numeric = float|int
SCREEN_DIM = (800, 600)

class Vec2d:
    def __init__(self, x: float|int, y) -> None:
        self.x = x
        self.y = y

    def __add__(self, other: Self) -> Self:
        return type(self)(self.x + other.x, self.y + other.y)

    def __sub__(self, other: Self) -> Self:
        return type(self)(self.x - other.x, self.y - other.y)

    def __mul__(self, other: Numeric) -> Self:
        return type(self)(self.x * other, self.y * other)
    
    __rmul__ = __mul__

    def __true_div__(self, other: Numeric) -> Self:
        return type(self)(self.x / other, self.y / other)
    
    def __matmul__ (self, other: Self) -> float:
        return self.x * other.x + self.y * other.y
    
    def __len__(self):
        return (self @ self)**0.5
    
    def int_pair(self):
        return int(self.x), int(self.y)


class Polyline:
    def __init__(self) -> None:
        self.points: list[Vec2d] = []
        self.velocities: list[Vec2d] = []

    def add(self, point: Vec2d, velocity: Optional[Vec2d] = None):
        if velocity is None:
            velocity = Vec2d(random.random() * 2, random.random() * 2)
        self.points.append(point)
        self.velocities.append(point)

    def get(self, index: int) -> tuple[Vec2d, Vec2d]:
        if index >= len(self.points):
            raise KeyError
        return self.points[index], self.velocities[index]

    def set_points(self):
        for p in range(len(self.points)):
            self.points[p] = self.points[p] + self.velocities[p]
            if self.points[p].x > SCREEN_DIM[0] or self.points[p].x < 0:
                self.velocities[p] = Vec2d(-self.velocities[p].x, self.velocities[p].y)
            if self.points[p].y > SCREEN_DIM[1] or self.points[p].y < 0:
                self.velocities[p] = Vec2d(self.velocities[p].x, -self.velocities[p].y)

    def draw_points(self, display: pygame.Surface, style="points", width=3, color=(255, 255, 255)):
        """функция отрисовки точек на экране"""
        if style == "line":
            for p_n in range(-1, len(self.points) - 1):
                pygame.draw.line(display, color,
                                self.points[p_n].int_pair(),
                                self.points[p_n+1].int_pair(), width)

        elif style == "points":
            for p in self.points:
                pygame.draw.circle(display, color,p.int_pair(), width)

class Knot(Polyline):
    def get_point(self, alpha, deg=None):
        if deg is None:
            deg = len(self.points) - 1
        if deg == 0:
            return self.points[0]
        return (self.points[deg] * alpha) + (self.get_point(alpha, deg - 1) * (1 - alpha))

    def get_points(self, count):
        alpha = 1 / count
        res = []
        for i in range(count):
            res.append(self.get_point(i * alpha))
        return res

    def get_knot(self, count):
        if len(self.points) < 3:
            return []
        res = []
        for i in range(-2, len(self.points) - 2):
            ptn = []
            ptn.append((self.points[i] + self.points[i + 1]) * 0.5)
            ptn.append(self.points[i + 1])
            ptn.append((self.points[i + 1] + self.points[i + 2]) * 0.5)

            res.extend(self.get_points(count))
        return res

    def add(self, point: Vec2d, velocity: Vec2d | None = None):
        return super().add(point, velocity)


Применение классов в рабочей программе см. в файле `./files/screen_new.py`